<a href="https://colab.research.google.com/github/super-onigiri-man/JOEU-FM/blob/main/FM_%E3%83%99%E3%82%B9%E3%83%88%E3%83%92%E3%83%83%E3%83%88%E3%83%A9%E3%83%B3%E3%82%AD%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FM愛媛 ベストヒットランキング ランキング早見表の使い方**



1.   左上のOpen in Colabを押してGoogle Colabを起動させます
2.   右のタブから明屋書店のデータを入れます。

　　　（このページを閉じるとこのページにある明屋書店のデータも消えます）
3.   コード左上の再生ボタン「上から」押す（コードが実行されます）
4.   途中で見たい週を選択します（1:今週,2:先週,3:選択した週）

5
.   1,2の場合は約30秒で結果が出力されます。

     （年末年始明けなどのランキングの日付が特殊な場合は3を選択してください）

     （水曜日の14時以降にランキングがサイトに掲載されるため、水曜日の正午以前に1を選択するとエラーが出ます）

     （日付がたまにバグるときがあります。日付を確認してください）

6
.   3の場合はYYYY-MM-DDの順序で入力してください。

     （例:2023年11月28日→2023-11-28）

     （これを選択した場合、入力された一番近い発表日のランキングが出ます）

     （未来の日付を入力するとエラーになります）

    

**利用上の注意！**

・このコードを書き換えないでください。書き換えると動かなくなる可能性があります。

  （将来の僕より優秀なプログラマーのADの方、このクソコードのおリファクタリングよろしくお願いします）

・連続で使用する場合は最低でも3分以上時間を開けてください。

  （連続でアクセスすると各サイトのサーバーが攻撃とみなし、アクセス制限をかけられる場合があります。）

・このシステムはリンクや明屋書店のExcelのフォーマットが変わると動かなくなります。

勉強になるかわかりませんが、このコードに説明を書いています。参考にでもしてください。

言語:Python,SQLite

使用した技術:スクレイピング（Webサイトから情報を取得してくる技術のこと）


プロセス：

処理に必要なデータのインポートおよびデータベースの作成

↓

各サイトからランキングを取得し、配列に書き込み

↓

配列からデータベースに情報を書き込み

↓

データベースにあるScoreの高い順にデータを出力

https://qiita.com/ku_a_i/items/9c68e8aed3c2c066bc60

In [8]:
!pip install openpyxl #Excelデータの読み書き
!pip install mojimoji #全角文字を半角文字にするためのモジュール

In [9]:
import sqlite3 #データベースを動かすためのモジュール
import pandas as pd #ランキングを作成するためのモジュール
import openpyxl #Excelから情報を取ってくるモジュール
import mojimoji #全角文字を半角文字に変換するためのモジュール



In [ ]:
dbname = ('test.db')#データベース名.db拡張子で設定
conn = sqlite3.connect(dbname, isolation_level=None)#データベースを作成、自動コミット機能ON
cursor = conn.cursor() #カーソルオブジェクトを作成
sql = """CREATE TABLE music_master (
    Title TEXT,
    Artist TEXT,
    Score DOUBLE,
    Last_Rank INT,
    Last_Number INT,
    On_Chart INT,
    PRIMARY KEY (Title, Artist)
);"""
#↑DBのフォーマット設定（別のところに書いておきます）
cursor.execute(sql)#executeコマンドでSQL文を実行
conn.commit()#データベースにコミット(Excelでいう上書き保存。自動コミット設定なので不要だが一応・・)

In [10]:
import requests
from bs4 import BeautifulSoup #スクレイピング（取得）用
import datetime #日付計算用

#scoreを元にしたオリコンシングル・オリコンデジタル・ビルボード総合ランキング

# オリコン週間ランキング用
OriconWeekData = []
# オリコンデジタルランキング用
OriconDigitalData = []
# ビルボードランキング用
BillboardData = []
# 明屋書店ランキング用
HaruyaData = []


def selecter():

    print("見たい週を選択してください")
    select = int(input("1:今回 2:前回 3:別の回"))
    return select

def OriconTodays():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    #print(d_today, todayweek)

    #オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    #日付は来週の水曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):# 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):# 今日が水曜日
        Oriconday = d_today + datetime.timedelta(days=5)
    elif (todayweek == 3):# 今日が木曜日
        Oriconday = d_today + datetime.timedelta(days=4)
    elif (todayweek == 4):# 今日が金曜日
        Oriconday = d_today + datetime.timedelta(days=3)
    elif (todayweek == 5):# 今日が土曜日
        Oriconday = d_today + datetime.timedelta(days=2)
    elif (todayweek == 6):# 今日が日曜日
        Oriconday = d_today + datetime.timedelta(days=1)

    return Oriconday


def OriconLastWeek():
    # 今日の日付を求める
    d_today = datetime.date.today()
    # 今日の曜日を求める
    todayweek = datetime.date.today().weekday()
    # print(d_today, todayweek)

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (todayweek == 0):  # 今日が月曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today
    elif (todayweek == 1):  # 今日が火曜日(先週(今週月曜日)のランキング表示)
        Oriconday = d_today - datetime.timedelta(days=1)
    elif (todayweek == 2):  # 今日が水曜日
        Oriconday = d_today - datetime.timedelta(days=2)
    elif (todayweek == 3):  # 今日が木曜日
        Oriconday = d_today - datetime.timedelta(days=3)
    elif (todayweek == 4):  # 今日が金曜日
        Oriconday = d_today - datetime.timedelta(days=4)
    elif (todayweek == 5):  # 今日が土曜日
        Oriconday = d_today - datetime.timedelta(days=5)
    elif (todayweek == 6):  # 今日が日曜日
        Oriconday = d_today - datetime.timedelta(days=6)

    return Oriconday


def OriconSelectWeek():
    # 日付の入力を促す
    date = input("2020年8月3日以降の日付を入力してください (YYYY-MM-DD): ")

    # 入力された日付をdatetimeオブジェクトに変換
    dt = datetime.datetime.strptime(date, "%Y-%m-%d")
    dt = dt.date()
    # 曜日を取得
    weekday = dt.weekday()

    # オリコンの発表は毎週水曜日のため、火曜日までは先週のランキングを表示
    # 日付は来週の月曜日付となる。
    if (weekday == 0):  # 今日が月曜日
        Oriconday = dt
    elif (weekday == 1):  # 今日が火曜日
        Oriconday = dt - datetime.timedelta(days=1)
    elif (weekday == 2):  # 今日が水曜日
        Oriconday = dt - datetime.timedelta(days=2)
    elif (weekday == 3):  # 今日が木曜日
        Oriconday = dt - datetime.timedelta(days=3)
    elif (weekday == 4):  # 今日が金曜日
        Oriconday = dt - datetime.timedelta(days=4)
    elif (weekday == 5):  # 今日が土曜日
        Oriconday = dt - datetime.timedelta(days=5)
    elif (weekday == 6):  # 今日が日曜日
        Oriconday = dt - datetime.timedelta(days=6)

    return Oriconday


def OriconWeekRank(Oriconday):#オリコン週間ランキング

    #1位から10位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2',class_='title') #曲名
    artist = soup.find(class_="content-rank-main").find_all('p',class_='name') #アーティスト名
    score = 6.0 #独自スコア
    rank = 1 #ランキング
    print(str(Oriconday) + "付けオリコン週間シングルランキング")


    for link, artist in zip(links, artist):
      # 10位までランクインしたものを配列に保存
      OriconWeekData.append([link.text,artist.text,"{:.1f}".format(score)])


      # 壊れたときの表示用
      # if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
      #   print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      # else: #10位（２ケタの場合）なら
      #   print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      rank = rank + 1
      score = score - 0.3


    #11位から20位
    load_url = "https://www.oricon.co.jp/rank/js/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')  # 曲名
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
      OriconWeekData.append([link.text,artist.text,"{:.1f}".format(score)])
        # 壊れたときの表示用
      # print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
      rank = rank + 1
      score = score - 0.3

    # print(OriconWeekRank)



def OriconDigitalRank(Oriconday):#オリコンデジタルシングルランキング
    # 1位から10位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    rank = 1
    score = float(6.0)
    print(str(Oriconday) + "付けオリコン週間デジタルシングルランキング")
    for link, artist in zip(links, artist):
      # 壊れたときの表示用
        # if rank != 10:#10位以下（１ケタの場合）なら（点数の位置を揃えるため）
        #     print(" " + str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        # else: #10位（２ケタの場合）なら
        #     print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        OriconDigitalData.append([link.text,artist.text,"{:.1f}".format(score)])
        rank = rank + 1
        score = score - 0.3

    # 11位から20位
    load_url = "https://www.oricon.co.jp/rank/dis/w/" + str(Oriconday) + "/p/2/"
    html = requests.get(load_url)
    soup = BeautifulSoup(html.text, "html.parser")
    links = soup.find(class_="content-rank-main").find_all('h2', class_='title')
    artist = soup.find(class_="content-rank-main").find_all('p', class_='name')  # アーティスト名
    for link, artist in zip(links, artist):
        OriconDigitalData.append([link.text,artist.text,"{:.1f}".format(score)])
        # print(str(rank) + "位 " + "{:.1f}　 ".format(score) + link.text + "/" + artist.text)
        rank = rank + 1
        score = score - 0.3

def BillboadRank(Oriconday):#ビルボードJAPAN HOT100ランキング

    # オリコンの日付とビルボードの発表日の差を埋めるための計算
    Billday = Oriconday - datetime.timedelta(days=5)

    print(str(Billday) + "付けビルボードJAPAN HOT100ランキング")

    #URL(ここを変更すると読み込まなくなります)
    url = 'https://www.billboard-japan.com/charts/detail?a=hot100&year='+str(Oriconday.year)+'&month='+str(Oriconday.month)+'&day='+str(Oriconday.day)
    #URLを取得してくる
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    songs = soup.find_all('p', class_='musuc_title') #曲名
    artists = soup.find_all('p', class_='artist_name') #アーティスト名
    score = 6.0 #基準点

    for i in range(20):#20回繰り替えす
      song = songs[i].text.strip()
      artist = artists[i].text.strip()
      BillboardData.append([song,artist,format(score, '.1f')])
      # if i < 9:
      #   print(f" {i+1}位: {format(score, '.1f')} {song} / {artist}") #1位から9位までのランキング
      # else:
      #   print(f"{i+1}位: {format(score, '.1f')} {song} / {artist}") #10位から20位までのランキング
      score = score - 0.3 #scoreを-0.3する


sel = selecter()
print(sel)

if sel==1: #今週のランキング表示
    OriconTodays()
    OriconWeekRank(OriconTodays())
    print()
    OriconDigitalRank(OriconTodays())
    print()
    BillboadRank(OriconTodays())

elif sel == 2: #前週のランキング表示
    OriconLastWeek()
    OriconWeekRank(OriconLastWeek())
    # print(OriconWeekData)
    print()
    OriconDigitalRank(OriconLastWeek())
    # print(OriconDigitalData)
    print()
    BillboadRank(OriconLastWeek())
    # print(BillboardData)

else : #別の週のランキング
    OriconWeekRank(OriconSelectWeek())
    print()
    OriconDigitalRank(OriconSelectWeek())
    print()
    BillboadRank(OriconSelectWeek())


見たい週を選択してください
1:今回 2:前回 3:別の回1
1
2024-01-15付けオリコン週間シングルランキング

2024-01-15付けオリコン週間デジタルシングルランキング

2024-01-10付けビルボードJAPAN HOT100ランキング


In [ ]:
#すでに存在するExcelワークブックを読み込む(絶対パス)
wb = openpyxl.load_workbook("1002ベストヒット愛媛.xlsx")
#ワークブック内のアクティブなシートを取得
ws = wb.active

HaruyaData = [[mojimoji.zen_to_han(ws["D4"].value, kana=False),mojimoji.zen_to_han(ws["C4"].value, kana=False),6.0],
              [mojimoji.zen_to_han(ws["D5"].value, kana=False),mojimoji.zen_to_han(ws["C5"].value, kana=False),5.7],
              [mojimoji.zen_to_han(ws["D6"].value, kana=False),mojimoji.zen_to_han(ws["C6"].value, kana=False),5.4],
              [mojimoji.zen_to_han(ws["D7"].value, kana=False),mojimoji.zen_to_han(ws["C7"].value, kana=False),5.1],
              [mojimoji.zen_to_han(ws["D8"].value, kana=False),mojimoji.zen_to_han(ws["C8"].value, kana=False),4.8],
              [mojimoji.zen_to_han(ws["D9"].value, kana=False),mojimoji.zen_to_han(ws["C9"].value, kana=False),4.5],
              [mojimoji.zen_to_han(ws["D10"].value, kana=False),mojimoji.zen_to_han(ws["C10"].value, kana=False),4.2],
              [mojimoji.zen_to_han(ws["D11"].value, kana=False),mojimoji.zen_to_han(ws["C11"].value, kana=False),3.9],
              [mojimoji.zen_to_han(ws["D12"].value, kana=False),mojimoji.zen_to_han(ws["C12"].value, kana=False),3.6],
              [mojimoji.zen_to_han(ws["D13"].value, kana=False),mojimoji.zen_to_han(ws["C13"].value, kana=False),3.3],
              [mojimoji.zen_to_han(ws["D14"].value, kana=False),mojimoji.zen_to_han(ws["C14"].value, kana=False),3.0],
              [mojimoji.zen_to_han(ws["D15"].value, kana=False),mojimoji.zen_to_han(ws["C15"].value, kana=False),2.7],
              [mojimoji.zen_to_han(ws["D16"].value, kana=False),mojimoji.zen_to_han(ws["C16"].value, kana=False),2.4],
              [mojimoji.zen_to_han(ws["D17"].value, kana=False),mojimoji.zen_to_han(ws["C17"].value, kana=False),2.1],
              [mojimoji.zen_to_han(ws["D18"].value, kana=False),mojimoji.zen_to_han(ws["C18"].value, kana=False),1.8],
              [mojimoji.zen_to_han(ws["D19"].value, kana=False),mojimoji.zen_to_han(ws["C19"].value, kana=False),1.5],
              [mojimoji.zen_to_han(ws["D20"].value, kana=False),mojimoji.zen_to_han(ws["C20"].value, kana=False),1.2],
              [mojimoji.zen_to_han(ws["D21"].value, kana=False),mojimoji.zen_to_han(ws["C21"].value, kana=False),0.9],
              [mojimoji.zen_to_han(ws["D22"].value, kana=False),mojimoji.zen_to_han(ws["C22"].value, kana=False),0.6],
              [mojimoji.zen_to_han(ws["D23"].value, kana=False),mojimoji.zen_to_han(ws["C23"].value, kana=False),0.3],
              ]


print(HaruyaData)

[['Dangerholic', 'Snow Man', 6.0], ['Make Up Day/Missing', 'なにわ男子', 5.7], ['アイドルなんかじゃなかったら', 'AKB48', 5.4], ['本音と建前', 'Sexy Zone', 5.1], ['SPECIALZ', 'King Gnu', 4.8], ['CREAK', 'SixTONES', 4.5], ['愛の花', 'あいみょん', 4.2], ['オレンジkiss', 'Snow Man', 3.9], ['春待ち鳥よ・・・', '岡ゆう子', 3.6], ['Mainstream', 'BE:FIRST', 3.3], ['哀傷歌/風を起こせ', '鳥羽一郎', 3.0], ['結-YOU-', 'ENHYPEN', 2.7], ['ブラザービート', 'Snow Man', 2.4], ['釧網本線', '走裕介', 2.1], ['鏡の花', '服部浩子', 1.8], ['海峡浪漫', '山内惠介', 1.5], ['心音(しんおん)', '中島みゆき', 1.2], ['UNFORGIVEN', 'LE SSERAFIM', 0.9], ['こっから', 'SixTONES', 0.6], ['THE IDOLM@STER MILLION THE@TER VARIETY 04', '我那覇響,横山奈緒,高坂海美,真壁瑞希,宮尾美也,三浦あずさ,二階堂千鶴,四条貴音,北上麗花,豊川風', 0.3]]


In [11]:
# 二次元配列にある情報をデータベースに登録
# オリコン週間ランキング
for entry in OriconWeekData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

In [12]:
# オリコンデジタルランキング
for entry in OriconDigitalData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

In [13]:
# ビルボードJAPAN
for entry in BillboardData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score,Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

In [ ]:
# 明屋書店
for entry in HaruyaData:
    title, artist, score = entry
    # 既存のデータがあればScoreを足して更新、なければ新規追加
    cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
        ON CONFLICT(Title, Artist) DO UPDATE SET Score = Score + ?
    ''', (title, artist, score, score))

# コミットして変更を保存
conn.commit()

**データベースから上位20個を出力する**

整形後もここの再生ボタンで出力してください

In [14]:
# Scoreの高い順に上位25件を取得
top_20_query = '''
    SELECT * FROM music_master
    WHERE Score IS NOT NULL AND Score != ''
    ORDER BY Score DESC
    LIMIT 20
'''
top_20_results = cursor.execute(top_20_query).fetchall()
# コミットして変更を保存
conn.commit()

# 整形後のデータ出力用
df = pd.read_sql('''
SELECT * FROM music_master
WHERE Score IS NOT NULL AND Score != ''
ORDER BY Score DESC
LIMIT 60''', conn)

df.head(20)


,Title,Artist,Score,Last_Rank,Last_Number,On_Chart
0,GOAT,Number_i,12.0,0,0,0
1,唱,Ado,11.4,2,2158,5
2,アイドル,YOASOBI,10.8,6,2158,25
3,第ゼロ感,10-FEET,7.2,19,2144,27
4,勇者,YOASOBI,6.9,3,2158,2
5,SPECIALZ,King Gnu,6.0,1,2158,6
6,HEARTBREAKER/C‘monova,Kis-My-Ft2,6.0,0,0,0
7,SOULSOUP,Official髭男dism,6.0,0,0,0
8,青のすみか,キタニタツヤ,5.7,7,2158,13
9,Monopoly,乃木坂46,5.7,0,0,0


**==========================ここまで=================================**

**ここから下はデータ整形用**

%はFM愛媛で言うレコード室の検索の＊と同じです

**指定したタイトルの行を削除する**

In [ ]:
# 削除したいTitleを入力してください（%は消さないでください（前方一致検索））
cursor.execute('''DELETE FROM music_master
WHERE Title LIKE
 'ランデヴー%';
 ''')


**指定したタイトルのScoreを任意の点数追加する**

In [ ]:
# 削除したいTitleとScoreの値を入力してください（%は消さないでください（前方一致検索））
cursor.execute('''UPDATE music_master
SET Score = Score +3.4
WHERE Title LIKE
'ANTE%';
 ''')

**指定したタイトルのScoreを+3点する**

In [ ]:
# 3点追加したいTitleを入力してください（%は消さないでください（前方一致検索））
cursor.execute('''UPDATE music_master
SET Score = Score + 3
WHERE Title LIKE
'（指定したタイトル）%';
 ''')

**データベースに曲名・アーティスト名・Scoreを書き込む**

明屋書店のExcel用（既にデータベースに曲がある場合は、上の任意の点数入力を用いること）

In [ ]:
# 一番下の行に必要事項を入力してください
cursor.execute('''
        INSERT INTO music_master (Title, Artist, Score, Last_Rank, Last_Number, On_Chart)
        VALUES (?, ?, ?, 0, 0, 0)
    ''', ('（曲名）', '（アーティスト名）', '（Scoreの値）'))

**データベース検索**

整備中

In [ ]:
cursor.execute('''
SELECT * FROM music_master
WHERE Title LIKE 'わたしの%'
   OR Artist LIKE 'F%';
    ''')



**マジカルサイト送信用Excel自動生成**

In [34]:
from openpyxl.styles import Font, PatternFill
from openpyxl import load_workbook
from openpyxl.styles.alignment import Alignment

# Excelファイルを読み込み
workbook = load_workbook('ベストヒットランキング テスト.xlsx')
sheet = workbook.active

# sheet.unmerge_cells('A1:F25')

# 今回のベストヒットランキング回数の値を設定
print("今回のベストヒットランキングの回数を入力してください")
this_rank_number = int(input())
rank = 1
# 見出しのNo.と日付
sheet.cell(row=3, column=2).value = "Ｎｏ."+str(+this_rank_number) #No.
sheet.cell(row=3, column=6).value = str(OriconTodays().year)+"年"+str(OriconTodays().month)+"月"+str(OriconTodays().day)+"日"#日付
# ランキングセルにデータを書き込み
for idx in range(1, len(top_20_results)*2, 2):
    result = top_20_results[idx // 2]  # idxを2で割った商を使って結果を取得
    title, artist, score, last_rank, last_number, on_chart= result

    # ランキング初登場曲
    # Last_Rankが空文字もしくはnullの場合、文字色を赤に変更
    if last_rank is None or last_rank == "" or last_rank == 0:
        sheet.cell(row=idx+5, column=2).value = rank
        sheet.cell(row=idx+5, column=2).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="FF0000") #ThisWeek
        sheet.alignment = Alignment(horizontal = 'center',vertical='center')
        sheet.merge_cells(start_row = idx +5 ,start_column=2 ,end_row = idx +6 , end_column = 2)
        sheet.cell(row=idx+5, column=3).value = "初"
        sheet.cell(row=idx+5, column=3).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="FF0000") #LastWeek
        sheet.merge_cells(start_row = idx +5 ,start_column=3 ,end_row = idx +6 , end_column = 3)
        sheet.cell(row=idx+5, column=4).value = "1"
        sheet.cell(row=idx+5, column=4).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="FF0000") #Onchart
        sheet.merge_cells(start_row = idx +5 ,start_column=4 ,end_row = idx +6 , end_column = 4)
        sheet.cell(row=idx+5, column=5).value = title
        sheet.cell(row=idx+5, column=6).value = artist

        rank += 1

    # ランキング再登場曲
    # Last_NumberがThis_Numberより2以上差がある場合、セルに「再」を入力
    elif (last_number - this_rank_number) >= 2 :
        sheet.cell(row=idx+5, column=2).value = rank#ThisWeek
        sheet.cell(row=idx+5, column=2).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=2 ,end_row = idx +6 , end_column = 2)
        sheet.cell(row=idx+5, column=3).value = "再"#LastWeek
        sheet.cell(row=idx+5, column=3).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=3 ,end_row = idx +6 , end_column = 3)
        sheet.cell(row=idx+5, column=4).value = on_chart+1
        sheet.cell(row=idx+5, column=4).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=4 ,end_row = idx +6 , end_column = 4)
        sheet.cell(row=idx+5, column=5).value = title
        sheet.cell(row=idx+5, column=6).value = artist

        rank+=1

    else:
    #  先週も登場した曲
    # セルにデータを書き込み
        sheet.cell(row=idx+5, column=5).value = title
        sheet.cell(row=idx+5, column=6).value = artist
        sheet.cell(row=idx+5, column=2).value = rank
        sheet.merge_cells(start_row = idx +5 ,start_column=2 ,end_row = idx +6 , end_column = 2)
        sheet.cell(row=idx+5, column=2).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.cell(row=idx+5, column=3).value = last_rank
        sheet.cell(row=idx+5, column=3).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=3 ,end_row = idx +6 , end_column = 3)
        # sheet.cell(row=idx, column=7).value = last_number
        sheet.cell(row=idx+5, column=4).value = on_chart+1
        sheet.cell(row=idx+5, column=4).font = Font(name ="BIZ UDPゴシック",size = 16,bold=True,color="000000")
        sheet.merge_cells(start_row = idx +5 ,start_column=4 ,end_row = idx +6 , end_column = 4)
        rank+=1

# Excelファイルを保存
workbook.save('ベストヒットランキング テスト２.xlsx')

# データベース接続を閉じる
# conn.close()

今回のベストヒットランキングの回数を入力してください
2121


**終了後はここから下を実行**

必ずランキングが作成されたことを確認すること！

**（別の回を出力している場合はここは絶対に実行しないこと！）**

In [ ]:
  # 今回のベストヒットランキングの回数を入力

for idx, result in enumerate(top_20_results, start=1):
    title, artist, score, _, _, _ = result
    cursor.execute('''
        UPDATE music_master
        SET Last_Rank = ?, On_Chart = On_Chart + 1, Last_Number = ?
        WHERE Title = ? AND Artist = ?
    ''', (idx, this_rank_number, title, artist))

cursor.execute('UPDATE music_master SET Score = 0;') #Scoreの値を全消し


In [ ]:
import csv

with open('楽曲データ.csv', 'w', newline='') as f:
    # CSVライターオブジェクトを作成
    csv_writer = csv.writer(f)

    # データを取得
    cursor.execute("SELECT * FROM music_master")
    rows = cursor.fetchall()

    # データをCSVに書き込む
    for row in rows:
        csv_writer.writerow(row)

from google.colab import files
files.download('楽曲データ.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**=====================================ここまで（ここから下は管理者用）=====================================**

**質が良すぎるデータづくり用**

2159回から

In [ ]:
import csv
import sqlite3

dbname = ('test.db')#データベース名.db拡張子で設定
conn = sqlite3.connect(dbname, isolation_level=None)#データベースを作成、自動コミット機能ON
cursor = conn.cursor() #カーソルオブジェクトを作成
sql = """CREATE TABLE music_master (
    Title TEXT,
    Artist TEXT,
    Score DOUBLE,
    Last_Rank INT,
    Last_Number INT,
    On_Chart INT,
    PRIMARY KEY (Title, Artist)
);"""
#↑DBのフォーマット設定（別のところに書いておきます）
cursor.execute(sql)#executeコマンドでSQL文を実行
conn.commit()#データベースにコミット(Excelでいう上書き保存。自動コミット設定なので不要だが一応・・)
with open('楽曲データ.csv', 'r') as f:
    # CSVリーダーオブジェクトを作成
    csv_reader = csv.reader(f)
    for row in csv_reader:
        # テーブルにデータを挿入
        cursor.execute("INSERT INTO music_master VALUES (?,?,?,?,?,?)", (row[0], row[1],row[2],row[3],row[4],row[5]))

In [ ]:
# Excelファイルの読み込み
excel_file = '3ベストヒットえひめランキングHP用.xlsx'
workbook = openpyxl.load_workbook(excel_file)
sheet = workbook.active


# データの処理と挿入
for row in range(6, 45, 2):
    title = mojimoji.zen_to_han(sheet['E' + str(row)].value, kana=False)
    artist = mojimoji.zen_to_han(sheet['F' + str(row)].value, kana=False)
    rank = sheet['B' + str(row)].value
    on_chart = sheet['D'+str(row)].value
    this_number = mojimoji.zen_to_han(sheet['B3'].value,kana=False)
    this_number = this_number.replace('No.','')

    if "再" in str(sheet['C' + str(row)].value) or "圏外" in str(sheet['C' + str(row)].value) :
        last_number = int(this_number) - 1
    elif "初" in str(sheet['C' + str(row)].value):
        on_chart = 1
        last_number = this_number
    else:
        last_number = this_number

    # データベースに挿入
    cursor.execute('''INSERT INTO music_master
     (Title, Artist, Score, Last_Rank, Last_Number, On_chart)
     VALUES (?, ?, 0.0, ?, ?, ?)
     ON CONFLICT(Title, Artist) DO UPDATE SET Last_Number = ?,Last_Rank = ?,On_Chart = ?''',
            (title, artist, rank, last_number, on_chart,last_number,rank,on_chart))

# コミットとクローズ
conn.commit()

FileNotFoundError: [Errno 2] No such file or directory: '3ベストヒットえひめランキングHP用.xlsx'

In [ ]:
df = pd.read_sql('''SELECT * FROM music_master ORDER BY Last_Number DESC;''',conn)
df.head(60)

,Title,Artist,Score,Last_Rank,Last_Number,On_Chart
0,アイドル,YOASOBI,0.0,6,2158,25
1,青のすみか,キタニタツヤ,0.0,7,2158,13
2,SPECIALZ,King Gnu,0.0,1,2158,6
3,Dangerholic,SnowMan,0.0,5,2158,5
4,唱,Ado,0.0,2,2158,5
5,Make Up Day,なにわ男子,0.0,11,2158,3
6,本音と建前,Sexy Zone,0.0,8,2158,3
7,アイドルなんかじゃなかったら,AKB48,0.0,18,2158,2
8,勇者,YOASOBI,0.0,3,2158,2
9,Anytime Anywhere,milet,0.0,13,2158,2


In [ ]:
cursor.execute('UPDATE music_master SET Score = 0;') #Scoreの値を全消し

In [ ]:
import csv

with open('楽曲データ.csv', 'w', newline='') as f:
    # CSVライターオブジェクトを作成
    csv_writer = csv.writer(f)

    # データを取得
    cursor.execute("SELECT * FROM music_master")
    rows = cursor.fetchall()

    # データをCSVに書き込む
    for row in rows:
        csv_writer.writerow(row)

from google.colab import files
files.download('楽曲データ.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>